# Step 2 Database Engineering

In [1]:
# Dependencies and installations
!rm hawaii.sqlite
!pip install pymysql
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

In [2]:
#Use the engine and connection string to create a database called hawaii.sqlite
engine = create_engine("sqlite:///hawaii.sqlite")

In [3]:
# Create a connection to the engine called `conn`
conn = engine.connect()

In [4]:
#Use declarative_base and create ORM classes for each table.
from sqlalchemy import Column, Float, Integer, String
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

In [5]:
#Use declarative_base and create ORM classes for each table.
class Measurement(Base):
    __tablename__ = 'measurement'
    id = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(String)
    prcp = Column(Float)
    tobs = Column(Integer)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"
    
class Station(Base):
    __tablename__ = 'station'
    id = Column(Integer, primary_key =True)
    station = Column(String)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [6]:
#create the tables in the database using create_all
from sqlalchemy.orm import Session
Base.metadata.create_all(engine)
session = Session(bind=engine)

In [7]:
#reading csv files
measurement_df = pd.read_csv('clean_hawaii_measurements.csv')
station_df= pd.read_csv('clean_hawaii_stations.csv')

In [8]:
# Use Orient='records' to create a list of data to write
# to_dict() cleans out DataFrame metadata as well
measurement_data = measurement_df.to_dict(orient='records')
station_data = station_df.to_dict(orient='records')

In [9]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [10]:
# Save references
Measurement = sqlalchemy.Table('measurement', metadata, autoload=True)
Station = sqlalchemy.Table('station', metadata, autoload=True)

In [11]:
# Use `table.delete()` to remove any existing data.
# Note that this is a convenience function so that you can re-run the example code multiple times.
# You would not likely do this step in production.
conn.execute(Measurement.delete())
conn.execute(Station.delete())

In [12]:
# Use `table.insert()` to insert the data into the table
conn.execute(Measurement.insert(), measurement_data)
conn.execute(Station.insert(), station_data)

In [14]:
#testing
conn.execute("select * from measurement limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0.0, 63),
 (3, 'USC00519397', '2010-01-03', 0.0, 74),
 (4, 'USC00519397', '2010-01-04', 0.0, 76),
 (5, 'USC00519397', '2010-01-07', 0.06, 70)]

In [15]:
conn.execute("select * from station").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6),
 (6, 'USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 21.33556, -157.71139, 19.5),
 (7, 'USC00519281', 'WAIHEE 837.5, HI US', 21.45167, -157.84888999999995, 32.9),
 (8, 'USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 21.3152, -157.9992, 0.9),
 (9, 'USC00516128', 'MANOA LYON ARBO 785.2, HI US', 21.3331, -157.8025, 152.4)]